# Agent Based Model: Main Notebook

In [820]:
import numpy.random as rnd
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Importing The Study Groups that are to be Tested

In [821]:
homogenous = pd.read_csv("Conditions/Homogenous.csv")
fiftyfifty = pd.read_csv("Conditions/FiftyFifty.csv")
heterogenous = pd.read_csv("Conditions/Heterogenous.csv")


for i in (range(len(homogenous.index))):
    print(list(homogenous.iloc[i][1:5]))


['ESFJ', 'ESFJ', 'ESFJ', 'ESFJ']
['ESTJ', 'ESTJ', 'ESTJ', 'ESTJ']
['ISFJ', 'ISFJ', 'ISFJ', 'ISFJ']
['ISTJ', 'ISTJ', 'ISTJ', 'ISTJ']
['ENFP', 'ENFP', 'ENFP', 'ENFP']
['ENFJ', 'ENFJ', 'ENFJ', 'ENFJ']
['INFP', 'INFP', 'INFP', 'INFP']
['INFJ', 'INFJ', 'INFJ', 'INFJ']
['ENTP', 'ENTP', 'ENTP', 'ENTP']
['ENTJ', 'ENTJ', 'ENTJ', 'ENTJ']
['INTP', 'INTP', 'INTP', 'INTP']
['INTJ', 'INTJ', 'INTJ', 'INTJ']
['ESFP', 'ESFP', 'ESFP', 'ESFP']
['ESTP', 'ESTP', 'ESTP', 'ESTP']
['ISFP', 'ISFP', 'ISFP', 'ISFP']
['ISTP', 'ISTP', 'ISTP', 'ISTP']


## Creating Study Groups

#### Practical Functions

In [822]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(1.5, 1.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000000)

In [823]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    Name_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = []
    
    for student in studygroup:
        Name_list.append(student.Name)
        extraversion_list.append(student.ExScore)
        sensing_list.append(student.SeScore) 
        thinking_list.append(student.ThScore)
        judging_list.append(student.JuScore)
        academic_list.append(student.Academic_Skill)

    data = pd.DataFrame({'Name': Name_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list})
    
    return data
    

#### Create Study Group

In [824]:
## Defining the Students ##
class Student():
    def __init__(self, Name, Ex, Se, Th, Ju):
        self.Name  = Name

        ## Personality Traits ## 
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension

        ## Personality Scores calculated with the personality() function##
        self.ExScore = 0
        self.SeScore = 0
        self.ThScore = 0
        self.JuScore = 0
        
        self.Scores = [] #list of all personality scores

        ## Academic Skills ##
        self.Academic_Skill = 0

        ## Own Solution ##
        self.Ind_Solution = []

def personality(student):
    # Extraversion vs. Introversion
    if student.Ex == "E":
        student.ExScore = max_vals[0]
        del max_vals[0]
    else:
        student.ExScore = min_vals[0]
        del min_vals[0]
    
    # Sensing vs. Intuition
    if student.Se == "S":
        student.SeScore = max_vals[0]
        del max_vals[0]
    else:
        student.SeScore = min_vals[0]
        del min_vals[0]
    
    # Thinking vs. Feeling
    if student.Th == "T":
        student.ThScore = max_vals[0]
        del max_vals[0]
    else:
        student.ThScore = min_vals[0]
        del min_vals[0]

    # Judging vs. Perceiving
    if student.Ju == "J":
        student.JuScore = max_vals[0]
        del max_vals[0]
    else:
        student.JuScore = min_vals[0]
        del min_vals[0]
    
    student.Scores = [student.ExScore, student.SeScore, student.ThScore, student.JuScore]

def skills(student):
    student.Academic_Skill = (1-student.SeScore)*0.40 + student.ThScore*0.20 + (1-student.JuScore)*0.40

In [825]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []
    names = ["Alfa", "Bravo", "Charlie", "Delta"]

    for i in range(len(student_list)):
        student = Student(names[i], student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        personality(student)
        skills(student)
        studygroup.append(student)
    
    return data_collect(studygroup, student_list), studygroup

## Creating the ABM

### Generating True Solution

In [826]:
def true_solution_generator(n_part_exercises, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    true_solution = []
    for i in range(n_part_exercises):
        true_solution.append(random.randint(range_elements[0], range_elements[1]))

    return true_solution, range_elements

### Generating Individual Solutions

In [827]:
def individual_solutions_generator(studygroup, true_solution, range_elements):
    '''
    #function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''
    
    all_solutions = []

    for student in studygroup[1]: # loop for each individual
        Ind_Solution_lst = []
        Ind_Solution_attr_lst = []
        
        for i in range(len(true_solution)): # loop for each part-exercise
            coin_toss = np.random.binomial(1, (student.Academic_Skill*0.7) + 0.3, 1)[0] # biased-coin flip
            if coin_toss == 1:
                Ind_Solution_lst.append(true_solution[i])
                Ind_Solution_attr_lst.append(true_solution[i])
            else:
                Ind_Solution_lst.append(random.randint(range_elements[0], range_elements[1]))
                Ind_Solution_attr_lst.append(random.randint(range_elements[0], range_elements[1]))
        
        all_solutions.append(Ind_Solution_lst)
        student.Ind_Solution = Ind_Solution_attr_lst
    
    return all_solutions

## Collaborative Problem Solving

##### Workflow notes
1. Who presents their solution e.g. agent A presents their solution to agent B, C, D -> THE PROPOSED SOLUTION
    -> Based on Extraversion score (Highest extraversion score is the most likely to present their solution)
2. According to an Agreeableness score (Social score for now) of the other agents (and maybe a Trustworthiness score of agent proposing), agents will update their solution
3. The solutions of the agents will be checked, if all they agree, this is their final solution. If not, the process will be repeated for a max of X ticks. If the groups do not converge, an accuracy score will still be calculated. 

In [828]:
def collaborative_solution(studygroup, max_ticks):
    max_ticks = max_ticks #turns in the simulation
    n_ticks = 0
    
    # extracting all 4 students
    Alfa = studygroup[1][0]
    Bravo = studygroup[1][1]
    Charlie = studygroup[1][2]
    Delta = studygroup[1][3]

    student_list = [Alfa, Bravo, Charlie, Delta]
    individual_solutions = [Alfa.Ind_Solution, Bravo.Ind_Solution, Charlie.Ind_Solution, Delta.Ind_Solution]

    while (n_ticks != max_ticks):
        # Starting a Round
        presenter_name = random.choices([Alfa.Name, Bravo.Name, Charlie.Name, Delta.Name], weights = [Alfa.ExScore, Bravo.ExScore, Charlie.ExScore, Delta.ExScore], k = 1)[0] # selecting the presenter of the round based on weighted random draw from extraversion scores
        
        Proposed_Solution = eval(presenter_name).Ind_Solution
        #print(n_ticks, presenter_name, Proposed_Solution)

        for student in student_list:
            if student == eval(presenter_name):
                pass
            for i in range(len(Proposed_Solution)): # looping through all part-exercises and evaluating against proposed solution.
                coin_toss = np.random.binomial(1, ((1-student.ThScore)*0.60 + eval(presenter_name).Academic_Skill*0.10), 1)[0] #high social skill has a greater chance of accepting the proposal and if the proposer has higher academic skill.
                if coin_toss == 1:
                    student.Ind_Solution[i] = Proposed_Solution[i]
                else:
                    student.Ind_Solution[i] = student.Ind_Solution[i]

        n_ticks += 1 #adding a tick to the simulation

        if all(x==individual_solutions[0] for x in individual_solutions): # initializes a break of while-loop if consensus has been reached (i.e. all solutions are the same)
            break



    #print("Number of iterations: " + str(n_ticks))
    return individual_solutions[0], n_ticks # returns their collective solution


In [829]:
def solution_evaluator(proposed_solution, true_solution):
    correct_part_exercise = 0
    wrong_part_exercise = 0

    for i in range(len(true_solution)):
        if proposed_solution[i] == true_solution[i]:
            correct_part_exercise += 1
        else:
            wrong_part_exercise += 1

    return correct_part_exercise / (correct_part_exercise + wrong_part_exercise)


In [830]:
def adaptation_degree(individual_solution, group_solution):
    '''
    #function to calculate the adaptation degree of the agents
    '''
    adapted = 0
    not_adapted = 0

    for i in range(len(individual_solution)):
        if individual_solution[i] == group_solution[i]:
            not_adapted += 1
        else:
            adapted += 1
    
    adaptation = adapted / (adapted + not_adapted)

    return adaptation

In [831]:
def synergy_calculator(individual_accuracies, group_accuracy):
    '''
    #function to calculate the gain of the agents
    '''
    avg_of_indi_accuracy = np.mean(individual_accuracies)
    max_of_indi_accuracy = np.max(individual_accuracies)
    
    weak_synergy = group_accuracy - avg_of_indi_accuracy
    strong_synergy = group_accuracy - max_of_indi_accuracy

    return weak_synergy, strong_synergy

# TEST RUN

In [849]:
# Selecting Personality Types
student_list_test = ["INTP", "INTP", "INTP", "INTP"]

In [850]:
# Creating the Study Group
studygroup1 = StudyGroup(student_list_test)

In [851]:
# Seeing their Generated Personality Scores
print(studygroup1[0])

      Name  type       E/I       S/N       T/F       J/P  Academic
0     Alfa  INTP  0.377614  0.438108  0.898946  0.154318  0.742819
1    Bravo  INTP  0.265280  0.316230  0.894770  0.151858  0.791719
2  Charlie  INTP  0.151305  0.103751  0.552213  0.366619  0.722295
3    Delta  INTP  0.374239  0.312633  0.555143  0.381013  0.633570


In [856]:
# Generating a True Solution
true_solution_test = true_solution_generator(10, [1, 3])

In [857]:
# Displaying the True Solution
true_solution_test[0]

[1, 1, 2, 2, 2, 2, 3, 2, 3, 3]

In [858]:
# Generating Individual Solutions
all_solutions = individual_solutions_generator(studygroup1, true_solution_test[0], true_solution_test[1])
all_solutions

[[1, 1, 2, 2, 2, 2, 3, 2, 2, 1],
 [1, 1, 1, 2, 1, 2, 3, 2, 3, 1],
 [1, 1, 2, 2, 2, 2, 3, 2, 3, 3],
 [1, 1, 2, 2, 2, 2, 3, 2, 2, 1]]

In [859]:
# Printing Individual Solutions
for student in studygroup1[1]:
    print(student.Name, student.Ind_Solution)

Alfa [1, 1, 2, 2, 2, 2, 3, 2, 3, 3]
Bravo [1, 1, 2, 2, 3, 2, 3, 2, 3, 1]
Charlie [1, 1, 2, 2, 1, 2, 3, 2, 1, 3]
Delta [1, 1, 2, 3, 2, 2, 3, 2, 3, 3]


In [860]:
# Printing Indivudal Accuracies
for student in studygroup1[1]:
    print(student.Name, solution_evaluator(student.Ind_Solution, true_solution_test[0]))

Alfa 1.0
Bravo 0.8
Charlie 0.8
Delta 0.9


In [861]:
# Creating Group Solution
group_solution = collaborative_solution(studygroup1, 100)

In [862]:
print(group_solution)
print(all_solutions)

([1, 1, 2, 2, 2, 2, 3, 2, 3, 3], 19)
[[1, 1, 2, 2, 2, 2, 3, 2, 2, 1], [1, 1, 1, 2, 1, 2, 3, 2, 3, 1], [1, 1, 2, 2, 2, 2, 3, 2, 3, 3], [1, 1, 2, 2, 2, 2, 3, 2, 2, 1]]


In [863]:
# Printing Group Accuracy
accuracy = solution_evaluator(group_solution[0], true_solution_test[0])
print(accuracy)

1.0


In [864]:
adaptation_degree(all_solutions[0], group_solution[0])

0.2

## ABM Model

In [844]:
def exercise_run(student_list: list, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    '''
    function to run one simulation of a study group completing the exercise (from individual to group solution)
    '''

    #### ---- LISTS ---- #### 
    ## ---- group level ---- ###
    group_accuracy_list = []
    n_tick_list = []
    weak_synergy_list = []
    strong_synergy_list = []

    ## ---- individual level ---- ###
    alfa_accuracy_list = []
    beta_accuracy_list = []
    charlie_accuracy_list = []
    delta_accuracy_list = []

    alfa_adaptation_list = []
    beta_adaptation_list = []
    charlie_adaptation_list = []
    delta_adaptation_list = []

    # ---- starting simulation ---- #
    for i in range(n_simulations):
        ## giving the study group personality values and skills ##
        studygroup = StudyGroup(student_list)
        
        ## ---- SOLUTIONS ---- ##
        ## generating the true solution ## 
        true_solution = true_solution_generator(n_part_exercises, range_solution)

        ## generating individual solutions ##
        all_solutions = individual_solutions_generator(studygroup, true_solution[0], true_solution[1])

        ## evaluating own solutions ##        
        alfa_accuracy = solution_evaluator(all_solutions[0], true_solution[0])
        beta_accuracy = solution_evaluator(all_solutions[1], true_solution[0])
        charlie_accuracy = solution_evaluator(all_solutions[2], true_solution[0])
        delta_accuracy = solution_evaluator(all_solutions[3], true_solution[0])

        all_accuracies = [alfa_accuracy, beta_accuracy, charlie_accuracy, delta_accuracy]
        
        alfa_accuracy_list.append(alfa_accuracy)
        beta_accuracy_list.append(beta_accuracy)
        charlie_accuracy_list.append(charlie_accuracy)
        delta_accuracy_list.append(delta_accuracy)

        ## generating group solution ##
        group_solution = collaborative_solution(studygroup, max_ticks)
        n_tick_list.append(group_solution[1]) # appending the number of ticks
        
        ## ---- MEASURES ---- ##
        ## calculating degree of adaptation ## 
        alfa_adaption = adaptation_degree(all_solutions[0], group_solution[0])
        beta_adaption = adaptation_degree(all_solutions[1], group_solution[0])
        charlie_adaption = adaptation_degree(all_solutions[2], group_solution[0])
        delta_adaption = adaptation_degree(all_solutions[3], group_solution[0])

        alfa_adaptation_list.append(alfa_adaption)
        beta_adaptation_list.append(beta_adaption)
        charlie_adaptation_list.append(charlie_adaption)
        delta_adaptation_list.append(delta_adaption)

        ## calculating the accuracy of the group solution ##
        group_accuracy = solution_evaluator(group_solution[0], true_solution[0])
        group_accuracy_list.append(group_accuracy) #appending the group accuracies

        # calculating the gain from the individual solutions to the group solution
        weak_synergy, strong_synergy = synergy_calculator(all_accuracies, group_accuracy)
        weak_synergy_list.append(weak_synergy)
        strong_synergy_list.append(strong_synergy)
        

    

    ## calculating the mean and standard deivations ##
    # ---- group level ---- #
    avg_group_accuracy = np.mean(group_accuracy_list)
    std_group_accuracy = np.std(group_accuracy_list)
    
    avg_group_n_tick = np.mean(n_tick_list)
    std_group_n_tick = np.std(n_tick_list)

    avg_weak_synergy  = np.mean(weak_synergy_list)
    std_weak_synergy = np.std(weak_synergy_list)

    avg_strong_synergy  = np.mean(strong_synergy_list)
    std_strong_synergy = np.std(strong_synergy_list)
    
    # ---- individual level ---- #
    avg_alfa_accuracy = np.mean(alfa_accuracy_list)
    std_alfa_accuracy = np.std(alfa_accuracy_list)

    avg_beta_accuracy = np.mean(beta_accuracy_list)
    std_beta_accuracy = np.std(beta_accuracy_list)

    avg_charlie_accuracy = np.mean(charlie_accuracy_list)
    std_charlie_accuracy = np.std(charlie_accuracy_list)

    avg_delta_accuracy = np.mean(delta_accuracy_list)
    std_delta_accuracy = np.std(delta_accuracy_list)

    avg_alfa_adaptation = np.mean(alfa_adaptation_list)
    avg_beta_adaptation = np.mean(beta_adaptation_list)
    avg_charlie_adaptation = np.mean(charlie_adaptation_list)
    avg_delta_adaptation = np.mean(delta_adaptation_list)
    
    ## dictionaries ## 
    exercise_run_dict_group = {
    'alfa': student_list[0], 
    'beta':student_list[1], 
    'charlie':student_list[2], 
    'delta':student_list[3], 
    'avg_accuracy': avg_group_accuracy, 
    'std_accuracy': std_group_accuracy, 
    'avg_n_tick': avg_group_n_tick, 
    'std_n_tick': std_group_n_tick, 
    'avg_weak_synergy': avg_weak_synergy,
    'std_weak_synergy': std_weak_synergy, 
    'avg_strong_synergy': avg_strong_synergy,
    'std_strong_synergy': std_strong_synergy}

    exercise_run_list_ind = list(zip(student_list, 
                                    [student_list, student_list, student_list, student_list], 
                                    [avg_alfa_accuracy, avg_beta_accuracy, avg_charlie_accuracy, avg_delta_accuracy],
                                    [std_alfa_accuracy, std_beta_accuracy, std_charlie_accuracy, std_delta_accuracy], 
                                    [avg_alfa_adaptation, avg_beta_adaptation, avg_charlie_adaptation, avg_delta_adaptation]))
    
    return exercise_run_dict_group, exercise_run_list_ind

In [845]:
def abm_model(condition, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    condition_group_df = pd.DataFrame(columns = ['alfa', 'beta', 'charlie', 'delta', 'avg_accuracy', 'std_accuracy', 'avg_n_tick', 'std_n_tick', 'avg_weak_synergy', 'std_weak_synergy', 'avg_strong_synergy', 'std_strong_synergy'])
    condition_ind_df = pd.DataFrame(columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        
    for i in (range(len(condition.index))):
        student_list = list(condition.iloc[i][1:5])
        
        ## Getting data frames for the group and individual levels ##
        exercise_run_dict_group, exercise_run_list_ind = exercise_run(student_list, max_ticks, n_part_exercises, range_solution, n_simulations) #creating dictionary
        
        exercise_run_group_df = pd.DataFrame(exercise_run_dict_group, index = [i]) #creating dataframe
        condition_group_df = pd.concat([condition_group_df, exercise_run_group_df], ignore_index = True) #concatenating dataframes
        
        exercise_run_ind_df = pd.DataFrame(exercise_run_list_ind, columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        condition_ind_df = pd.concat([condition_ind_df, exercise_run_ind_df], ignore_index = True)

    return condition_ind_df, condition_group_df


In [846]:
homogenous_ind_df, homogeous_group_df = abm_model(condition = homogenous, 
        max_ticks = 200, 
        n_part_exercises = 10, 
        range_solution = [1, 3], 
        n_simulations = 100
        )

In [847]:
sorted_wsynergy_homogenous = homogeous_group_df.sort_values("avg_weak_synergy", ascending = True)
sorted_wsynergy_homogenous

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy
5,ENFJ,ENFJ,ENFJ,ENFJ,0.729,0.137328,6.63,3.028713,-0.01300,0.133580,-0.161,0.143454
7,INFJ,INFJ,INFJ,INFJ,0.734,0.129012,7.22,2.696590,-0.00950,0.139811,-0.152,0.145931
9,ENTJ,ENTJ,ENTJ,ENTJ,0.775,0.149248,21.82,13.011825,-0.00725,0.144061,-0.139,0.147577
4,ENFP,ENFP,ENFP,ENFP,0.815,0.136657,5.42,2.713595,-0.00500,0.133651,-0.128,0.140769
12,ESFP,ESFP,ESFP,ESFP,0.737,0.148091,6.87,2.903980,0.00175,0.145999,-0.130,0.153948
14,ISFP,ISFP,ISFP,ISFP,0.741,0.134235,6.98,3.023177,0.00500,0.128695,-0.136,0.136763
8,ENTP,ENTP,ENTP,ENTP,0.869,0.111081,14.24,7.926058,0.00600,0.097540,-0.098,0.106752
1,ESTJ,ESTJ,ESTJ,ESTJ,0.720,0.144914,24.24,14.954678,0.00725,0.146982,-0.136,0.159700
2,ISFJ,ISFJ,ISFJ,ISFJ,0.679,0.160807,7.53,3.578980,0.00775,0.165253,-0.144,0.183478
11,INTJ,INTJ,INTJ,INTJ,0.797,0.141035,18.93,10.872217,0.00800,0.132706,-0.111,0.139925


In [848]:
fifty_ind, fifty_group = abm_model(condition = fiftyfifty, 
        max_ticks = 200, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

KeyboardInterrupt: 

In [ ]:
sorted_wsynergy_fifty = fifty_group.sort_values("avg_weak_synergy", ascending = True)
sorted_wsynergy_fifty

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy
4,ESTJ,ESTJ,INFP,INFP,0.654,0.145203,14.77,7.861113,-0.04200,0.132471,-0.206,0.154156
11,ENFJ,ENFJ,INFP,INFP,0.681,0.142615,6.52,2.475803,-0.03125,0.127494,-0.204,0.158063
12,ESTJ,ESTJ,ISFP,ISFP,0.619,0.163521,14.83,7.803916,-0.01575,0.141229,-0.167,0.160034
0,ESFJ,ESFJ,INTP,INTP,0.687,0.177570,14.88,8.469097,-0.00750,0.163764,-0.215,0.185674
2,ESFP,ESFP,INTJ,INTJ,0.694,0.144097,15.36,9.494756,-0.00325,0.130080,-0.155,0.141686
5,ENFP,ENFP,ISTJ,ISTJ,0.689,0.150927,13.94,7.181671,0.00100,0.135919,-0.179,0.149195
8,ESFJ,ESFJ,ISTP,ISTP,0.634,0.151803,17.49,11.507819,0.01450,0.142266,-0.154,0.147932
13,ENFP,ENFP,INTJ,INTJ,0.752,0.126079,12.11,6.117017,0.01600,0.124926,-0.129,0.142334
7,ENTJ,ENTJ,ISFP,ISFP,0.715,0.132947,12.30,5.700000,0.02225,0.117095,-0.144,0.124354
10,ESFP,ESFP,INTP,INTP,0.778,0.128515,14.46,12.046095,0.03050,0.116436,-0.132,0.124804


In [ ]:
hetero_ind, hetero_group = abm_model(condition = heterogenous, 
        max_ticks = 200, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [ ]:
sorted_wsynergy_hetero = hetero_group.sort_values("avg_weak_synergy", ascending = True)
sorted_wsynergy_hetero

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy
10,ENFP,INTJ,INFP,ENTJ,0.725,0.119478,11.60,5.761944,-0.02050,0.111489,-0.151,0.129225
9,ESTJ,ISFP,ISTJ,ESFP,0.626,0.155319,17.13,9.802709,-0.01725,0.141010,-0.188,0.173943
14,ESTJ,ISTJ,ENFJ,INFJ,0.599,0.155881,17.25,10.518912,-0.01650,0.159029,-0.181,0.165345
0,ESFJ,ESTJ,ENFP,INFJ,0.636,0.152656,10.49,4.353148,-0.01175,0.142805,-0.185,0.158351
5,ESTJ,ESTP,ISTP,INTJ,0.675,0.154515,25.21,13.281035,-0.00950,0.139856,-0.163,0.165925
2,ISTJ,ISTP,ESFP,ENTJ,0.668,0.162407,18.37,10.181999,-0.00950,0.146193,-0.166,0.159512
12,ESFJ,ISFJ,ENFP,INFP,0.674,0.166505,7.28,2.724261,-0.00800,0.148614,-0.196,0.163658
6,ENFP,ISFP,ESFP,INFP,0.719,0.144010,7.32,2.999600,0.01200,0.123414,-0.149,0.143871
13,ENTP,INTP,ESFP,ISFP,0.749,0.131526,12.59,7.195964,0.01250,0.128865,-0.139,0.142755
4,ESFJ,ISFJ,ISTJ,INTP,0.652,0.175203,17.12,11.746727,0.01400,0.169496,-0.201,0.181381


In [ ]:
homogenous_mean_acc = np.mean(homogeous_group_df['avg_accuracy'])
fifty_mean_acc = np.mean(fifty_group['avg_accuracy'])
hetero_mean_acc = np.mean(hetero_group['avg_accuracy'])

homogenous_std_acc = np.std(homogeous_group_df['avg_accuracy'])
fifty_std_acc = np.std(fifty_group['avg_accuracy'])
hetero_std_acc = np.std(hetero_group['avg_accuracy'])

homogenous_mean_tick = np.mean(homogeous_group_df['avg_n_tick'])
fifty_mean_tick = np.mean(fifty_group['avg_n_tick'])
hetero_mean_tick = np.mean(hetero_group['avg_n_tick'])

homogenous_std_tick = np.std(homogeous_group_df['avg_n_tick'])
fifty_std_tick = np.std(fifty_group['avg_n_tick'])
hetero_std_tick = np.std(hetero_group['avg_n_tick'])

print(tuple([homogenous_mean_acc, fifty_mean_acc, hetero_mean_acc]))
print(tuple([homogenous_std_acc, fifty_std_acc, hetero_std_acc]))

print(tuple([homogenous_mean_tick, fifty_mean_tick, hetero_mean_tick]))
print(tuple([homogenous_std_tick, fifty_std_tick, hetero_std_tick]))

(0.7756250000000001, 0.7101875, 0.69575)
(0.057832597857955505, 0.055735108717486145, 0.0530112016464445)
(12.939375, 13.895624999999999, 14.46125)
(6.637924627424976, 3.0645533295041547, 5.041543507448885)
